In [141]:
from collections import namedtuple
import locale
import os

import pandas as pd


if os.name == 'nt':
    locale.setlocale(locale.LC_NUMERIC, 'Portuguese_Brazil.1252')
else:
    locale.setlocale(locale.LC_NUMERIC, ('pt_BR', 'UTF-8'))

In [ ]:
# fonte_igp_di = 'http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=1411282625' # ao ano
# http://www.yahii.com.br/igpdi.html
# https://www.debit.com.br/tabelas/tabela-completa.php?indice=igp
# http://www14.fgv.br/fgvdados20/default.aspx
# https://calculador.com.br/calculo/correcao-valor-por-indice

In [ ]:
# Fonte FGVDados: http://www14.fgv.br/fgvdados20/default.aspx

In [3]:
df_fgv = pd.read_excel('../dados-brutos/xgvxConsulta-igp-di.xls',
                   skiprows=13,
                   usecols=[0, 1],
                   names=['ano_mes', 'igp_di'],
                   parse_dates=[0])
df_fgv.ano_mes = df_fgv.ano_mes.dt.to_period('M')
df_fgv.igp_di = df_fgv.igp_di.rolling(2).apply(lambda s: s.iloc[1] / s.iloc[0]).fillna(0)

In [7]:
# Fonte IPEADATA
fonte_igp_di = 'http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=39615' # ao mês

In [8]:
df_ipea = pd.read_html(fonte_igp_di,
                       decimal=',',
                       thousands='.',
                       header=0,
                       parse_dates=[0],
                       encoding='utf-8')[2]
df_ipea.columns = ['ano_mes', 'igp_di']
df_ipea.ano_mes = df_ipea.ano_mes.dt.to_period('M')
df_ipea.igp_di = df_ipea.igp_di / 100 + 1

In [9]:
# comparar as duas fontes, checando erros
(df_fgv.set_index('ano_mes').igp_di.round(4)[1:] - df_ipea.set_index('ano_mes').igp_di).pipe(lambda s: s[s > 1e-14])

Series([], Freq: M, Name: igp_di, dtype: float64)

Series([], Freq: M, Name: igp_di, dtype: float64)

In [10]:
df_fgv.to_excel('../dados/igp-di.xlsx', index=False)

In [65]:
df_igp_di = pd.read_excel('../dados/igp-di.xlsx', parse_dates=[0])
df_igp_di.ano_mes = df_igp_di.ano_mes.dt.to_period('M')

In [ ]:
# Fonte: https://www.bcb.gov.br/content/acessoinformacao/museudocs/pub/SintesePadroesMonetariosBrasileiros.pdf
# 1942 	Out 	Cruzeiro 	Cr$ 	Rs 1$000 (um mil réis)
# 1967 	Fev 	Cruzeiro Novo 	NCr$ 	Cr$ 1.000,00 (um mil cruzeiros)
# 1970 	Mai 	Cruzeiro 	Cr$ 	NCr$ 1,0 (um cruzeiro novo)
# 1986 	Fev 	Cruzado 	Cz$ 	Cr$ 1.000,00 (um mil cruzeiros)
# 1989 	Jan 	Cruzado Novo 	NCz$ 	Cz$ 1.000,00 (um mil cruzados)
# 1990 	Mar 	Cruzeiro 	Cr$ 	NCz$ 1,00 (um cruzado novo)
# 1993 	Ago 	Cruzeiro Real 	CR$ 	Cr$ 1.000,00 (um mil cruzeiros)
# 1994 	Jul 	Real 	R$ 	CR$ 2.750,00 (dois mil setecentos e cinquenta cruzeiros reais)

In [110]:
pad_mon = namedtuple('padrão_monetário', ['data', 'divisor', 'nome', 'simbolo'])
reis          = pad_mon(data=pd.Period('1600-01'), divisor=1, nome='Réis', simbolo='Rs')
cruzeiro      = pad_mon(data=pd.Period('1942-10'), divisor=1000, nome='Cruzeiro', simbolo='Cr$')
cruzeiro_novo = pad_mon(data=pd.Period('1967-02'), divisor=1000, nome='Cruzeiro Novo', simbolo='NCr$')
cruzeiro2     = pad_mon(data=pd.Period('1970-05'), divisor=1, nome='Cruzeiro', simbolo='Cr$')
cruzado       = pad_mon(data=pd.Period('1986-02'), divisor=1000, nome='Cruzado', simbolo='Cr$')
cruzado_novo  = pad_mon(data=pd.Period('1989-01'), divisor=1000, nome='Cruzado Novo', simbolo='NCr$')
cruzeiro3     = pad_mon(data=pd.Period('1990-03'), divisor=1, nome='Cruzeiro', simbolo='Cr$')
cruzeiro_real = pad_mon(data=pd.Period('1993-08'), divisor=1000, nome='Cruzeiro Real', simbolo= 'CR$')
real          = pad_mon(data=pd.Period('1994-07'), divisor=2750, nome='Real', simbolo='R$')
moedas = (cruzeiro, cruzeiro_novo, cruzeiro2, cruzado, cruzado_novo, cruzeiro3, cruzeiro_real, real)

In [139]:
def inflacionar(data_inicial,
                data_final,
                valor,
                incluir_data_inicial=False,
                incluir_data_final=True):
    '''Formato da data: "AAAA-MM"
    '''
    data_inicial = pd.Period(data_inicial)
    data_final = pd.Period(data_final)

    if not incluir_data_final:
        data_final -= - 1
    
    df_filtrada = df_igp_di[df_igp_di.ano_mes.between(data_inicial, data_final)]
    if not incluir_data_inicial:
        indice_correcao = df_filtrada.iloc[1:].igp_di.cumprod().iloc[-1]
    else: 
        indice_correcao = df_filtrada.igp_di.cumprod().iloc[-1]
        
    valor_corrigido = valor * indice_correcao
    indice_correcao_liquida = indice_correcao
    
    for moeda in moedas:
        if (df_filtrada.ano_mes == moeda.data).any():
            valor_corrigido /= moeda.divisor       
            indice_correcao_liquida /= moeda.divisor
            
    for moeda in moedas:
        if data_inicial > moeda.data:
            nome_moeda_inicial = moeda.nome
            simbolo_moeda_inicial = moeda.simbolo
        if data_final > moeda.data:
            nome_moeda_final = moeda.nome
            simbolo_moeda_final = moeda.simbolo
            
    print(f'Índice de variação bruta do perído: {locale.format_string("%.2f", indice_correcao, grouping=True)} \n'
          f'Índice de variação líquida do período: {locale.format_string("%.2f", indice_correcao_liquida, grouping=True)}\n'
          f'Valor a atualizar: {simbolo_moeda_inicial} {locale.format_string("%.2f", valor, grouping=True)} ({nome_moeda_inicial})\n'
          f'Valor atualizado:  {simbolo_moeda_final} {locale.format_string("%.2f", valor_corrigido, grouping=True)} ({nome_moeda_final})')
    return valor_corrigido, indice_correcao

In [140]:
inflacionar('1967-02', '1994-06', 1000)[0].round(2)

Índice de variação bruta do perído: 4.133.482.062.239,18 
Índice de variação líquida do período: 4,13
Valor a atualizar: Cr$ 1.000,00 (Cruzeiro)
Valor atualizado:  CR$ 4.133,48 (Cruzeiro Real)


4133.48